## Install Dependencies

In [ ]:
#!git clone https://github.com/GFDRR/caribbean-rooftop-classification.git
#!pip -q install -U segment-geospatial
#!pip -q install -U git+https://github.com/IDEA-Research/GroundingDINO.git

In [ ]:
#!pip -q install leafmap localtileserver huggingface_hub

In [ ]:
#%cd caribbean-rooftop-classification/
#!git checkout add/tutorials
#%cd tutorials

## Imports and Setup

In [ ]:
import os
import gdown

import matplotlib.pyplot as plt
import rasterio as rio
import geopandas as gpd
import leafmap

from rasterio.plot import show
from shapely.geometry import box
from samgeo import SamGeo

import sys
sys.path.insert(0, "../")
from utils import langsam_utils

%load_ext autoreload
%autoreload 2

## Download Image

In [ ]:
image_dir = "../data/rasters/drone/"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

filename = "drone_salisbury_cropped_DOM.tif"
image_file = os.path.join(image_dir, filename)

if not os.path.isfile(image_file):
    url = "https://drive.google.com/uc?id=1SBqJuGfCmrL9YXbwdoK3uOkbPSE4uFGJ"
    gdown.download(url, image_file, quiet=False)

## Load Image

In [ ]:
image = rio.open(image_file)
data = gpd.GeoDataFrame({"id":1,"geometry":[box(*image.bounds)]}, crs=image.crs)
data = data.to_crs("EPSG:4326")
bbox = data.geometry.values[0]
centroid = bbox.centroid

In [ ]:
map = leafmap.Map(center=[centroid.y, centroid.x], zoom=20)
map.add_raster(image_file, layer_name="Image", zoom=20)
map

## LangSAM

In [ ]:
model = langsam_utils.LangSAM()
text_prompt = "house"

In [ ]:
image_np, image_pil = langsam_utils.get_image_arrays(image_file)
masks, boxes, phrases, logit = model.predict(
    image_pil, text_prompt, box_threshold=0.3, text_threshold=0.3
)

In [ ]:
langsam_utils.visualize(image_np, image_pil, boxes, masks)

In [ ]:
out_file = "preds.gpkg"
langsam_utils.save_predictions(mask_overlay, out_file)

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
map.add_vector(out_file, layer_name="Vector", style=style)
map